# 1. Introduction
This notebook provides the Python code to create, train, and monitor a deep learning model for object detection. To achieve this, it employs the state-of-the-art You Only Look Once (YOLO) architecture. Compared to R-CNNs (see notebooks `object_detection_rcnn_custom.ipynb` and `object_detection_rcnn_standard.ipynb`), instead of applying the neural network at multiple locations and scales, YOLO processes the entire image with a single network pass, significantly reducing model complexity and inference time.

Unlike the other notebooks and use cases in TorchSuite, this one leverages a third-party training framework, [3 Lines of Code (3LC)](https://3lc.ai/), which has been specifically designed for training YOLO models and featuring a comprehensive web-based monitoring interface.

This notebook was originally developed by [Duality AI](https://www.duality.ai/) as a template for a Kaggle competition and was later adapted for a specific use case: detecting the presence of humans in images.

To use the 3LC framework, please follow the steps described in Section 2.

## 1.1. The DeepCount App

**TorchSuite** includes a proof-of-concept application called **DeepCount**, which uses the trained YOLO model to detect humans in images. Once the model is trained, it can be tested directly through this app.

You can find DeepCount in the subfolder: `demos/deep_count`.

# 2. 3LC setup
See the [3LC Quickstart](https://docs.3lc.ai/3lc/latest/quickstart/quickstart.html) for more details on the sections below.

## 2.1. Create a free 3LC account

Before getting started, make sure you have a free 3LC account. Go to https://3lc.ai and click "Sign Up" in the upper right corner. Sign up for a 3LC account (or log in to an existing one) using your preferred method. Then copy your API Key from the 3LC account home page.

## 2.2. Create a Python environment

Create a Python environment called "3LC" using your preferred tool:
- `python -m venv 3LC`
  - See the [Python venv documentation](https://docs.python.org/3.12/library/venv.html) for details
- `conda create -n 3LC`
  - See the [conda documentation](https://docs.conda.io/projects/conda/en/stable/user-guide/getting-started.html) for details, or reach out if you are having trouble

## 2.3. Install the 3LC YOLO integration

With your "3LC" Python environment activated, install the 3LC YOLO integration with the following command. This will also install the `3lc` Python package and all required dependencies.

```bash
pip install git+https://github.com/3lc-ai/3lc-ultralytics@develop
```
And install the Gradio GUI framework:

```bash
pip install gradio
```

## 2.4. Configure 3LC with your API key

Configure 3LC to use the API key you copied earlier (or get it [here](https://account.3lc.ai/api-key)).

```bash
3lc login <paste API key here>
```

## 2.5. Start the 3LC Object Service

The [3LC Object Service](https://docs.3lc.ai/3lc/latest/user-guide/object-service/index.html#object-service) is responsible for serving your dataset and metrics to the 3LC Dashboard. It is started from the terminal and can be terminated by pressing Q.

```bash
3lc service --no-public-examples
```

## 2.6. Launch the 3LC Dashboard

After starting the Object Service, launch the 3LC Dashboard in a browser at https://dashboard.3lc.ai and log in to your 3LC account. Your 3LC project will be displayed in the Dashboard once you begin creating 3LC data below. See the [3LC Dashboard](https://docs.3lc.ai/3lc/latest/user-guide/dashboard/index.html#dashboard-index) documentation for more details.

Note that if you want to browse example 3LC projects (in addition to your own), you can start the Object Service in the step above without specifying the `--no-public-examples` argument.

# 2.7. Verify 3LC Setup

Now we are almost ready to start creating 3LC `Table`s for your dataset, and `Run`s for your training runs with Ultralytics YOLO.

We first import 3LC to verify that the installation and configuration was successful. Make sure you have done the `3lc login` step above to avoid prompts or errors related to specifying a 3LC API key.

In [ ]:
import tlc

# 3. Downloading the Dataset

Download the People Detection dataset from the following link, selecting the **YOLOv8** annotations:

🔗 https://universe.roboflow.com/leo-ueno/people-detection-o4rdr/dataset/10

After downloading, update the `yolo_params.yml` file to include the paths to the training, validation, and test datasets.

# 4. Creating 3LC Tables

Run the following code to create 3LC Tables for the dataset. Before that, modify the paths for the training and validation sets.

In [ ]:
import os
import shutil

def scan_labels(labels_dir, images_dir, bad_dir):

    """
    Scans YOLO label files in `labels_dir` for invalid entries.
    Moves invalid label files and corresponding images to `bad_dir`.
    """
    
    os.makedirs(bad_dir, exist_ok=True)

    def is_label_file_valid(file_path):
        with open(file_path, "r") as f:
            for line_num, line in enumerate(f, start=1):
                parts = line.strip().split()
                if not parts:
                    print(f"[EMPTY] {file_path} (line {line_num})")
                    return False
                if len(parts) != 5:
                    print(f"[BAD] {file_path} (line {line_num} has {len(parts)} values)")
                    return False
                try:
                    int(parts[0])
                    [float(x) for x in parts[1:]]
                except ValueError:
                    print(f"[INVALID VALUE] {file_path} (line {line_num})")
                    return False
        return True

    # Scan all label files
    for file in os.listdir(labels_dir):
        if not file.endswith(".txt"):
            continue
        label_path = os.path.join(labels_dir, file)
        image_name = os.path.splitext(file)[0] + ".jpg"
        image_path = os.path.join(images_dir, image_name)

        if not is_label_file_valid(label_path):
            print(f"Moving bad label and image: {file}")
            shutil.move(label_path, os.path.join(bad_dir, file))
            if os.path.exists(image_path):
                shutil.move(image_path, os.path.join(bad_dir, image_name))
            else:
                print(f"[MISSING IMAGE] {image_name}")

    print("✅ Scan complete. Bad files moved to:", bad_dir)


# Paths for training set
labels_dir = r"D:\Repos\coco_dataset\People Detection.v10-rf-detr-nano.yolov8\train\labels" # Change the path
images_dir = r"D:\Repos\coco_dataset\People Detection.v10-rf-detr-nano.yolov8\train\images" # Change the path
bad_dir = os.path.join(r"D:\Repos\coco_dataset\People Detection.v10-rf-detr-nano.yolov8\train", "train_bad") # Change the path
scan_labels(labels_dir, images_dir, bad_dir)

# Paths for validation set
labels_dir = r"D:\Repos\coco_dataset\People Detection.v10-rf-detr-nano.yolov8\valid\labels" # Change the path 
images_dir = r"D:\Repos\coco_dataset\People Detection.v10-rf-detr-nano.yolov8\valid\images" # Change the path
bad_dir = os.path.join(r"D:\Repos\coco_dataset\People Detection.v10-rf-detr-nano.yolov8\valid", "valid_bad") # Change the path
scan_labels(labels_dir, images_dir, bad_dir)

In [ ]:
import tlc
PROJECT_NAME = "DeepCount"  # Place all 3LC Tables and Runs in the same project

# This for loop allows you to create multiple 3LC Tables (e.g., train and val sets) in one go
for split in ["train", "val"]:
    table = tlc.Table.from_yolo(
        dataset_yaml_file="yolo_params.yaml",  # the yolo_params.yaml file in the data folder you generate from Falcon
        split=split,
        table_name="initial",
        dataset_name=split,
        project_name=PROJECT_NAME,
    )

    print(f"Created table with URL: {table.url}")

# 5. Training a YOLO model

The following code trains a YOLO model with the 3LC YOLO integration.

In [ ]:
import torch
torch.cuda.empty_cache()

print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
!pip show torch

In [ ]:
from tlc_ultralytics import Settings, YOLO
import tlc

# Load YOLOv10-balanced
yolo_version = '10b'
RUN_NAME = f"run-{yolo_version}"  # Define the run name to organize all your runs in a nice way

# Set 3LC specific settings
settings = Settings(
    project_name=PROJECT_NAME,
    run_name=RUN_NAME,
    run_description=f"yolo {yolo_version}",
)

In [ ]:
# Update the URLs for the train and val tables when you make data revisions in 3LC Dashboard
train_table = tlc.Table.from_url("C:/Users/ssre_/AppData/Local/3LC/3LC/projects/DeepCount/datasets/train/tables/initial") # Change the path 
val_table = tlc.Table.from_url("C:/Users/ssre_/AppData/Local/3LC/3LC/projects/DeepCount/datasets/val/tables/initial") # Change the path

model = YOLO(F"yolov{yolo_version}.pt")

# Training configuration
model.train(
    tables={"train": train_table, "val": val_table},
    settings=settings,         # your hyperparameter settings dict
    imgsz=640,                 # scale shorter side to 640px (matches typical YOLO training)
    epochs=50,                 # enough to converge on 15k images
    batch=8,                   # you can adjust based on memory; effective batch = batch * accum_steps
    nbs=16,                    # nominal batch size for auto-scaling
    project=PROJECT_NAME,
    name=RUN_NAME,
    workers=1,                 # adjust to your CPU cores
    resume=False,              # continue training if checkpoint exists

    # Optimizer and learning rate
    optimizer="auto",          # default YOLO choice
    lr0=0.001,                 # slightly lower than default 0.01 for stability
    lrf=0.01,                  # final LR fraction (default)
    cos_lr=True,
    momentum=0.937,            # default for SGD
    weight_decay=0.0005,       # default

    # Warmup settings
    warmup_epochs=3,           # default
    warmup_bias_lr=0.1 * 0.006,
    warmup_momentum=0.8,       # default

    # Data augmentation (minimal since dataset already augmented)
    augment=False,
    mosaic=0.0,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    flipud=0.0,
    fliplr=0.5,
    degrees=10,
    translate=0.005,
    scale=0.0,
    perspective=0.0,
    cutmix=0.0,

    # Training control
    patience=100,        # early stopping patience
    agnostic_nms=True,   # class-agnostic NMS
    visualize=True,      # visualize during training
)

# 6. The DeepCount App

Once the model has been trained, move the exported model file to the following folder:

`demos/deep_count`

Next, navigate to that directory in your terminal and run the application using the following command:

```bash
python app.py --model <model_name>
```

This will launch the DeepCount app, allowing you to test the trained YOLO model on sample images.